In [5]:
import torch
import json
import pandas as pd

from aau2v.config import ModelConfig, TrainerConfig
from aau2v.dataset_center import load_dataset_center
from aau2v.trainer import PyTorchTrainer
from aau2v.model import load_model, PyTorchModel
from aau2v.util import set_seed

ModuleNotFoundError: No module named 'aau2v'

In [20]:
def calc_accuracy(
    model: PyTorchModel,
    num_item: int,
    test_dataset: list[tuple[int, list[int], list[int]]],
    top_k: list[int],
) -> dict[str, float]:
    model.eval()
    hit_counts = {k: 0 for k in top_k}
    seq_index = torch.LongTensor(list(map(lambda d: d[0], test_dataset)))
    context_items = torch.LongTensor(list(map(lambda d: d[1], test_dataset)))
    rec_list = model.output_rec_lists(
        seq_index=seq_index,
        item_indices=context_items,
        cand_item_indices=torch.arange(num_item),
        k=max(top_k),
    )
    for i in range(len(test_dataset)):
        for k in top_k:
            hit_counts[k] += len(set(test_dataset[i][2]) & set(rec_list[i][:k]))

    total_rec = len(test_dataset)
    results = {
        f"Accuracy@{k}": hit_count / total_rec / k
        for k, hit_count in hit_counts.items()
    }
    return results

In [25]:
run_configs = [
    {
        "name": "AAU2V (d=16)",
        "model_name": "aau2v",
        "d_model": 16,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V (d=32)",
        "model_name": "aau2v",
        "d_model": 32,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V (d=64)",
        "model_name": "aau2v",
        "d_model": 64,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V w/o WT",
        "model_name": "aau2v",
        "d_model": 64,
        "use_weight_tying": False,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V w/o AT",
        "model_name": "aau2v",
        "d_model": 64,
        "use_weight_tying": True,
        "use_attention": False,
        "use_meta": True,
    },
    {
        "name": "AAU2V w/o AI",
        "model_name": "aau2v",
        "d_model": 64,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": False,
    },
    {
        "name": "User2Vec (d=16)",
        "model_name": "user2vec",
        "d_model": 16,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "User2Vec (d=32)",
        "model_name": "user2vec",
        "d_model": 32,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "User2Vec (d=64)",
        "model_name": "user2vec",
        "d_model": 64,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
]

In [26]:
results = {}

for run_config in run_configs:
    set_seed(0)

    results[run_config["name"]] = {"accuracy": []}
    model_config = ModelConfig(
        max_embedding_norm=1,
        d_model=run_config["d_model"],
        use_attention=run_config["use_attention"],
        use_meta=run_config["use_meta"],
        use_weight_tying=run_config["use_weight_tying"],
    )
    trainer_config = TrainerConfig(
        dataset_name="movielens",
        lr=1e-4,
        model_name=run_config["model_name"],
        epochs=5,
    )

    print(model_config)
    print(trainer_config)

    dataset_center = load_dataset_center(
        dataset_name=trainer_config.dataset_name,
        window_size=model_config.window_size,
        data_dir="../data",
    )
    model = load_model(
        dataset_center=dataset_center,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer = PyTorchTrainer(
        model=model,
        dataset_center=dataset_center,
        trainer_config=trainer_config,
    )

    def on_iter_end(epoch: int, batch: int) -> None:
        if (batch + 1) % 1000 == 0:
            result = calc_accuracy(
                model=model,
                num_item=dataset_center.num_item,
                test_dataset=dataset_center.test_datasets["test"],
                top_k=[10, 30, 50, 100],
            )
            print(f"epoch: {epoch}, result: {result}")
            results[run_config["name"]]["accuracy"].append(result)

            with open("result.json", "w") as f:
                json.dump(results, f)

    losses = trainer.fit(on_iter_end=on_iter_end)
    results[run_config["name"]]["loss"] = losses

ModelConfig(d_model=16, init_embedding_std=0.2, max_embedding_norm=1, window_size=5, negative_sample_size=5, use_weight_tying=True, use_meta=True, use_attention=True)
TrainerConfig(model_name='user2vec', dataset_name='movielens', epochs=10, batch_size=64, lr=0.0001, weight_decay=0, verbose=False, model_dir='model', save_model=True, device='cpu')
num_seq: 6040, num_item: 3706, num_item_meta: 28, num_seq_meta: 30, num_item_meta_types: 3, num_seq_meta_types: 3
train: 589192 valid: 110547


 11%|█▏        | 1053/9207 [00:03<00:36, 225.08it/s]

epoch: 0, result: {'Accuracy@10': 0.02853551225644249, 'Accuracy@30': 0.03002304630211607, 'Accuracy@50': 0.031150219987429287, 'Accuracy@100': 0.03203016970458831}


 22%|██▏       | 2036/9207 [00:07<00:31, 230.46it/s]

epoch: 0, result: {'Accuracy@10': 0.028849780012570707, 'Accuracy@30': 0.030379216425728054, 'Accuracy@50': 0.03113764927718416, 'Accuracy@100': 0.032325581395348836}


...

loss for train: 0.0045516634


 62%|██████▏   | 1065/1728 [00:01<00:00, 671.85it/s]

epoch: 9, result: {'Accuracy@10': 0.06970458830923947, 'Accuracy@30': 0.07186256023465326, 'Accuracy@50': 0.07161533626649906, 'Accuracy@100': 0.07199245757385292}


100%|██████████| 1728/1728 [00:02<00:00, 816.28it/s]

loss for valid: 0.0046585808
saved model to model/user2vec-9.pt


In [6]:
with open("result.json", "r") as f:
    results = json.load(f)

top_k = [30, 50, 100]
data = {}
for method, result in results.items():
    data[method] = []
    for k in top_k:
        a = max(map(lambda r: r[f"Accuracy@{k}"], result["accuracy"]))
        data[method].append(a)

pd.options.display.precision = 4
df = pd.DataFrame(data).T
df.columns = [f"Accuracy@{k}" for k in top_k]
df

,Accuracy@30,Accuracy@50,Accuracy@100
AAU2V (d=16),0.2090,0.1935,0.1720
AAU2V (d=32),0.2108,0.1960,0.1759
AAU2V (d=64),0.2109,0.1985,0.1787
AAU2V w/o WT,0.2121,0.1987,0.1802
AAU2V w/o AT,0.2110,0.1984,0.1784
AAU2V w/o AI,0.2003,0.1916,0.1724
User2Vec (d=16),0.1839,0.1761,0.1591
User2Vec (d=32),0.1887,0.1802,0.1630
User2Vec (d=64),0.2014,0.1907,0.1726
